In [5]:
import torch
import torch.nn.functional as F

torch.manual_seed(27)

# Example tensor of shape (1, 1, H, W)
DIM = 4
W = torch.rand(DIM, DIM)
P1 = torch.rand(int(DIM/2), DIM)
P2 = torch.rand(DIM, int(DIM/2))



In [6]:
load_weights_path = "models/512.pt"
checkpoint = torch.load(load_weights_path)

In [ ]:
encoder.blocks.block_6.block.residual_attention.layer.attention.output_projection.output_projection.weight

In [9]:
list(checkpoint["model"].keys())

['embedding_layer.layers.0.weight',
 'embedding_layer.layers.1.layer.weight',
 'encoder.blocks.block_0.block.residual_attention.layer.pre_norm.weight',
 'encoder.blocks.block_0.block.residual_attention.layer.pre_norm.bias',
 'encoder.blocks.block_0.block.residual_attention.layer.attention.input_projection.weight',
 'encoder.blocks.block_0.block.residual_attention.layer.attention.output_projection.weight',
 'encoder.blocks.block_0.block.residual_feedforward.layer.pre_norm.weight',
 'encoder.blocks.block_0.block.residual_feedforward.layer.pre_norm.bias',
 'encoder.blocks.block_0.block.residual_feedforward.layer.feedforward.logging_ff_pre_relu.weight',
 'encoder.blocks.block_0.block.residual_feedforward.layer.feedforward.logging_ff_pre_relu.bias',
 'encoder.blocks.block_0.block.residual_feedforward.layer.feedforward.logging_ff_post_relu.weight',
 'encoder.blocks.block_0.block.residual_feedforward.layer.feedforward.logging_ff_post_relu.bias',
 'encoder.blocks.block_1.block.residual_attenti

: 

In [ ]:
raise

In [2]:
P1@W#@P2

tensor([[1.5872, 1.1626, 1.6426, 0.9742],
        [0.4341, 0.6854, 0.8121, 0.5155]])

In [3]:
DIM1 = 12
DIM2 = 4
W = torch.rand(DIM1, DIM2)
P1 = torch.rand(int(DIM1/2), DIM1)
P2 = torch.rand(DIM2, int(DIM2/2))

In [4]:
r = P1@W@P2
print(W.shape)
print(r.shape)
r

torch.Size([12, 4])
torch.Size([6, 2])


tensor([[5.6443, 5.0776],
        [6.5575, 5.7987],
        [7.7880, 7.0714],
        [7.1346, 6.4522],
        [6.2099, 5.7591],
        [7.6796, 6.9450]])

In [5]:
xb = 512
yb = 2048
xs = 256
ys = 1024
W = torch.rand(xb, yb)
P1 = torch.rand(xs, xb)
P2 = torch.rand(yb, ys)

In [6]:
r = P1@W@P2
print(W.shape)
print(r.shape)
r

torch.Size([512, 2048])
torch.Size([256, 1024])


tensor([[130796.1172, 128785.4453, 129663.6562,  ..., 128499.3594,
         130298.6484, 129971.3906],
        [133012.6562, 130936.7422, 131843.7656,  ..., 130729.7422,
         132476.1875, 132249.2500],
        [129721.0234, 127680.9062, 128570.0000,  ..., 127446.5781,
         129220.4062, 128920.0234],
        ...,
        [130213.5469, 128230.0234, 129112.5391,  ..., 127997.6562,
         129677.8828, 129393.8047],
        [130767.9375, 128707.3594, 129624.9375,  ..., 128489.2734,
         130267.7734, 129968.9062],
        [132819.5938, 130733.5156, 131616.2812,  ..., 130468.2188,
         132302.6875, 132004.3594]])

In [7]:
from lizrd.core.misc import Linear


l = Linear(
    4, #xs
    8, #xb
    init_type = "kaiming_uniform",
    init_scale = 1.0
)
l

Linear(in_features=4, out_features=8, bias=False)

In [8]:
l.weight

Parameter containing:
tensor([[-0.0854, -0.6050,  0.3916, -0.5043],
        [ 0.4044, -0.6925, -0.1916,  0.1707],
        [ 0.7449, -0.3332,  0.7467,  0.4768],
        [ 0.6021,  0.2765,  0.4113,  0.6645],
        [-0.1919, -0.2049,  0.1844, -0.7438],
        [ 0.4192, -0.2509, -0.2455,  0.0148],
        [ 0.7097, -0.5523,  0.6878, -0.3706],
        [-0.5385,  0.4877,  0.6276, -0.7681]], requires_grad=True)

In [ ]:
from lizrd.train.train_utils import get_model
from research.conditional.utils.model_utils import get_classes_from_module_names, get_ff_layer
from research.projected_distillation.llm import ProjectedAttention, ProjectedFeedForward
dmodel = 256
dff = 1024
projected_dmodel = dmodel*2
projected_dff = dff*2
n_blocks = 8
init_type = "kaiming_uniform"
init_scale= 1.0
block_modules = {
    "FF": lambda: ProjectedFeedForward(
            dmodel, dff, projected_dmodel, projected_dff, init_type=init_type, init_scale= init_scale
        ),
    "Attention": lambda: ProjectedAttention(
            dmodel=dmodel,
            projected_dmodel=projected_dmodel,
            heads=8,
            causal=True,
            dhead=None,
            flash=False,
            init_type=init_type,
            init_scale=init_scale,
        )
}

model = get_model(
        max_length=256,
        vocab_size=50048,
        block_modules=block_modules,
        dm=dmodel,
        n_blocks=n_blocks,
        device=(
            torch.device("cuda")
        ),  # in case of  DDP/FSDP, we initialize the model on CPU and move it to the GPU later
        init_type=init_type,
        init_scale=init_scale,
        ddp_enabled=None,
        fsdp_enabled=True,
        fsdp_param_precision=None,
        fsdp_mixed_precision_ignore_classes=None,
        fsdp_offload_params=None,
        fsdp_min_num_params=None,
        activation_checkpointing_modules=None,
        is_logging_process=True,
        fsdp_modules_to_wrap=get_classes_from_module_names("EmbeddingLayer,PredictionHead,TransformerBlock")
    )

RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

: 

In [20]:
for name, params in model.named_parameters():
    print(name)

embedding_layer.layers.0.weight
embedding_layer.layers.1.layer.weight
encoder.blocks.block_0.block.residual_FF.layer.pre_norm.weight
encoder.blocks.block_0.block.residual_FF.layer.pre_norm.bias
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_pre_relu_p11.weight
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_pre_relu_p11.bias
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_pre_relu.weight
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_pre_relu.bias
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_pre_relu_p12.weight
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_pre_relu_p12.bias
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_post_relu_p21.weight
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_post_relu_p21.bias
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_post_relu.weight
encoder.blocks.block_0.block.residual_FF.layer.FF.logging_ff_post_relu.bias
encoder.blocks.blo